In [20]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import seaborn as sns
import pickle

from tmu.models.classification.vanilla_classifier import TMClassifier
from sklearn.metrics import confusion_matrix
from src.lib.care import calculate_care_score

In [21]:
# Create folder figures_1 if it does not exist
os.makedirs("figures", exist_ok=True)

In [22]:
def load_test_dataset(farm, event_id):
    X = np.loadtxt(f"./data_test/X_{farm}_{event_id}.txt", dtype=np.uint32)
    X = np.array(X).astype(np.uint32)

    # Take the first 3000 rows
    #X = X[:3000]

    return X


def load_test_labels(farm, event_id):
    # Load dataframe from file
    df = pd.read_csv(f"./data_test/y_{farm}_{event_id}.csv")

    labels = df['label'].values
    status_ids = df['status_type_id'].values
    train_test = df['train_test'].values

    # Take the first 3000 rows
    #labels = labels[:3000]
    #status_ids = status_ids[:3000]
    #train_test = train_test[:3000]

    return np.array(labels).astype(np.uint32), np.array(status_ids).astype(np.uint32), train_test


def load_test_label(farm, event_id):
    event_info = pd.read_csv(f"../../../data/care_to_compare/Wind Farm {farm}/event_info.csv", delimiter=';')

    metadata = event_info[event_info['event_id'] == event_id]

    event_label = metadata["event_label"].values[0]

    return False if event_label == "anomaly" else True

In [23]:
def calculate_accuracy(labels, predictions):
    accuracy = np.sum(labels == predictions) / len(labels)
    return accuracy

In [24]:
def load_model(filename) -> TMClassifier:
    with open(filename, "rb") as f:
        model = pickle.load(f)

    return model

In [25]:
def plot_predictions(X, y, z, p, name):
    cm = confusion_matrix(y, p)

    sns.heatmap(cm, annot=True, fmt='d', cmap='YlOrBr',
                xticklabels=['Predicted normal', 'Predicted anomaly'],
                yticklabels=['Actual normal', 'Actual anomaly'])

    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')

    plt.savefig(f"./figures/confusion_matrix{name}.pdf")
    plt.close()


def get_predictions(X, y, z, name, tm):
    X_predictions = tm.predict(X)

    # Accuracy
    accuracy = calculate_accuracy(y, X_predictions)

    plot_predictions(X, y, z, X_predictions, name)

    return X_predictions, accuracy

In [26]:
# test_datasets = [19, 27, 77, 83, 52, 21, 2, 23, 87, 74, 86, 82]
test_datasets = [19, 27, 77, 2, 23, 87, 74, 86, 82]

tm_classifier = load_model("best_1.pkl")


def run_prediction(farm, dataset):
    X = load_test_dataset(farm, dataset)
    labels, status_ids, train_test = load_test_labels(farm, dataset)

    is_normal = load_test_label(farm, dataset)

    predictions, accuracy = get_predictions(X, labels, status_ids, f"{farm}_{dataset}", tm_classifier)

    return X, labels, status_ids, train_test, is_normal, predictions, accuracy


In [27]:
# Create a dataframe with status_type_id;label;prediction
elements = []

for set in test_datasets:
    res = run_prediction("B", set)

    result_df = pd.DataFrame({
        'status_type_id': res[2],
        'label': res[1],
        'prediction': res[5],
        'train_test': res[3],
    })

    print(f"Done with {set}. Accuracy: {res[6]}")

    elements.append({'dataset': set, 'normal': res[4], 'data': result_df, 'accuracy': res[6]})

2025-05-15 18:51:10,175 - matplotlib.backends.backend_pdf - DEBUG - Assigning font /F1 = '/System/Library/Fonts/Supplemental/Arial.ttf'
2025-05-15 18:51:10,193 - matplotlib.backends.backend_pdf - DEBUG - Embedding font /System/Library/Fonts/Supplemental/Arial.ttf.
2025-05-15 18:51:10,193 - matplotlib.backends.backend_pdf - DEBUG - Writing TrueType font.
Done with 19. Accuracy: 0.6429906542056075
2025-05-15 18:51:10,773 - matplotlib.backends.backend_pdf - DEBUG - Assigning font /F1 = '/System/Library/Fonts/Supplemental/Arial.ttf'
2025-05-15 18:51:10,785 - matplotlib.backends.backend_pdf - DEBUG - Embedding font /System/Library/Fonts/Supplemental/Arial.ttf.
2025-05-15 18:51:10,786 - matplotlib.backends.backend_pdf - DEBUG - Writing TrueType font.
Done with 27. Accuracy: 0.4780114722753346
2025-05-15 18:51:11,401 - matplotlib.backends.backend_pdf - DEBUG - Assigning font /F1 = '/System/Library/Fonts/Supplemental/Arial.ttf'
2025-05-15 18:51:11,411 - matplotlib.backends.backend_pdf - DEBUG 

In [28]:
# Safe the results to results.pkl
with open("results.pkl", "wb") as f:
    pickle.dump(elements, f)

In [30]:
score = calculate_care_score(elements)
print(score)

0.6384027854214592
